In [5]:
import os, sys
parent_dir = os.path.abspath('..')
if not parent_dir in sys.path:
    sys.path.append(parent_dir)

In [6]:
import json
from ultralytics import YOLO
from pascal import annotation_from_xml
from lib.images import read_image_file
from lib.file_utils import get_subdirectories, create_directory, list_images

In [7]:
IMAGES_DIR = "../images"
labels_map = {}
with open(f'{IMAGES_DIR}/class-names.txt') as f:
    for idx, line in enumerate(f):
        labels_map[line.strip()] = idx
labels_map

{'medical-mask': 0,
 'person-mask': 1,
 'person-masked-medical': 2,
 'person-masked-non-medical': 3}

In [8]:
def read_pascal_voc(image_path):
    end_index = image_path.rindex('.')
    xml_path = image_path[0:end_index] + '.xml'
    annotations = annotation_from_xml(xml_path)
    return annotations

In [9]:
IMAGE_TARGET_DIR = '../images-yolo'
create_directory(IMAGE_TARGET_DIR)

for dataset_type in ['test', 'train']:
    target_base_dir = f'{IMAGE_TARGET_DIR}/{dataset_type}'
    create_directory(target_base_dir)

    for class_dir in get_subdirectories(f'{IMAGES_DIR}/{dataset_type}'):
        images = list_images(f"{IMAGES_DIR}/{dataset_type}/{class_dir}")
        if dataset_type == 'train':
            images.extend(list_images(f"{IMAGES_DIR}-aug/{dataset_type}/{class_dir}"))

        print(f"Processing {len(images)} images in {dataset_type} dataset")
        
        for image_path in images:
            annotation = read_pascal_voc(image_path)
            img = read_image_file(image_path)
            
            # rename image file
            image_filename = class_dir.lower() + '-' + os.path.basename(image_path)
            
            img.save(f'{target_base_dir}/{image_filename}')
            yolo_annotation_path = f'{target_base_dir}/{image_filename[0:-4]}.txt'
            with open(yolo_annotation_path, 'w') as f:
                f.write(annotation.to_yolo(labels_map, precision=6))

Processing 6 images in test dataset
Processing 6 images in test dataset
Processing 6 images in test dataset
Processing 5 images in test dataset
Processing 6 images in test dataset
Processing 43 images in train dataset
Processing 53 images in train dataset
Processing 52 images in train dataset
Processing 39 images in train dataset
Processing 49 images in train dataset
